# German Mammals GBIF

Using the Data from GBIF to create the list of German Mammals

In [2]:
from pygbif import species as species
from pygbif import occurrences as occ

In [7]:
occ.search(classKey=359,country='DE', limit=10)

{'offset': 0,
 'limit': 10,
 'endOfRecords': False,
 'count': 767703,
 'results': [{'key': 5028885891,
   'datasetKey': 'aa6c5ee6-d4d7-4a65-a04f-379cffbf4842',
   'publishingOrgKey': '2754e9c0-0e43-4f65-968a-6f16b9c378ce',
   'installationKey': 'dcceb601-2fb0-49dc-9cd2-7c00056f2b2c',
   'hostingOrganizationKey': '2754e9c0-0e43-4f65-968a-6f16b9c378ce',
   'publishingCountry': 'DE',
   'protocol': 'BIOCASE',
   'lastCrawled': '2025-05-09T16:19:17.871+00:00',
   'lastParsed': '2025-05-09T16:32:59.595+00:00',
   'crawlId': 345,
   'extensions': {},
   'basisOfRecord': 'HUMAN_OBSERVATION',
   'occurrenceStatus': 'PRESENT',
   'taxonKey': 5220126,
   'kingdomKey': 1,
   'phylumKey': 44,
   'classKey': 359,
   'orderKey': 731,
   'familyKey': 5298,
   'genusKey': 2440927,
   'speciesKey': 5220126,
   'acceptedTaxonKey': 5220126,
   'scientificName': 'Capreolus capreolus (Linnaeus, 1758)',
   'acceptedScientificName': 'Capreolus capreolus (Linnaeus, 1758)',
   'kingdom': 'Animalia',
   'phylum